### Description: this script is going to visualize the streamflow (and resultant return periods) for all Hydrobasins in Kenya given a specific date as input.

#### Step 1: Import Libraries

In [3]:
import datetime
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import gcsfs

#### Step 2: Load Files

In [4]:
# Function for opening zarr datasets
def open_zarr(path):
  return xr.open_zarr(
      store=path,
      chunks=None,
      storage_options=dict(token='anon')
      )

In [5]:
# Set file paths
base_directory = 'gs://flood-forecasting/hydrologic_predictions/model_id_8583a5c2_v0/'
reanalysis_path = os.path.join(base_directory, 'reanalysis/streamflow.zarr/')
return_periods_path = os.path.join(base_directory, 'return_periods.zarr/')
# Load Hydrobasins
hydrobasin_path = os.path.join(os.pardir, 'data', 'Kenya_hybas_L12.shp')

In [6]:
# Load files
reanalysis_ds = open_zarr(reanalysis_path)
return_periods_ds = open_zarr(return_periods_path)
hydrobasins = gpd.read_file(hydrobasin_path)

print(f'{reanalysis_ds.sizes = }')
print(f'{return_periods_ds.sizes = }')

reanalysis_ds.sizes = Frozen({'gauge_id': 1031646, 'time': 16063})
return_periods_ds.sizes = Frozen({'gauge_id': 1031646})


#### Step 3: Filter datasets by Kenyan hydrobasins

In [7]:
# Extract all the basin IDs in our HydroBasin dataset
kenya_basin_ids = hydrobasins['HYBAS_ID'].unique()
kenya_basin_ids = [f"hybas_{basin_id}" for basin_id in kenya_basin_ids]
# Get the gauge IDs from the reanalysis dataset.
reanalysis_ids = reanalysis_ds.gauge_id.values.astype(str)
# Find common basin IDs
common_ids = np.intersect1d(kenya_basin_ids, reanalysis_ids)
# Filter the reanalysis dataset using only the common IDs
filtered_reanalysis_ds = reanalysis_ds.sel(gauge_id=common_ids)
# Filter the return period dataset using the same IDs
filtered_return_periods_ds = return_periods_ds.sel(gauge_id=common_ids)

#### Step 4: Extract data for a specific date

In [8]:
# What date?
selected_date = "2020-05-01"

In [9]:
# Extract streamflow data for the selected date
streamflow_da = filtered_reanalysis_ds.streamflow.sel(time=selected_date)
# Convert to dataframe
streamflow_df = streamflow_da.to_dataframe().reset_index()
# Do the same for the return period dataset
return_periods_df = filtered_return_periods_ds.to_dataframe().reset_index()

#### Step 5: Calculate return period given streamflow

In [10]:
# Define function for running operation
def estimate_return_period(streamflow, gauge_id, rp_df):
    '''
    Function takes as input a streamflow value and interpolates the resultant return period
    from the RP dataframe for that specific hydrobasin
    '''
    # Filter row corresponding to the gauge_id
    row = rp_df[rp_df['gauge_id'] == gauge_id]
    if row.empty:
        return np.nan
    
    # Get all columns that start with "return_period_"
    rp_columns = [col for col in row.columns if col.startswith("return_period_")]
    
    # Create lists for the return period values and streamflow thresholds
    rp_values = []
    flow_thresholds = []
    for col in rp_columns:
        # extract numeric part of column name
        try:
            rp = float(col.split("_")[-1])
        except ValueError:
            continue
        rp_values.append(rp)
        flow_thresholds.append(row[col].values[0])
    
    # Convert to numpy arrays
    flow_thresholds = np.array(flow_thresholds)
    rp_values = np.array(rp_values)
    
    # Sort by flow thresholds
    sort_idx = np.argsort(flow_thresholds)
    flow_thresholds = flow_thresholds[sort_idx]
    rp_values = rp_values[sort_idx]
    
    # Check if streamflow is below the smallest threshold or above highest threshold
    if streamflow <= flow_thresholds[0]:
        return 0 # if it is below 2-year RP we will report 0
    if streamflow >= flow_thresholds[-1]:
        return rp_values[-1] # going to assume it's largest RP (200)
    
    # Linear interpolation for all other cases
    estimated_rp = np.interp(streamflow, flow_thresholds, rp_values)
    
    return estimated_rp

In [11]:
# Map the function across the streamflow dataframe
streamflow_df['estimated_rp'] = streamflow_df.apply(
    lambda row: estimate_return_period(row['streamflow'], row['gauge_id'], return_periods_df), axis=1
)

#### Step 6: Merge dataframes

In [12]:
# Only want to keep a few columns from original one
hydrobasins_subset = hydrobasins[['HYBAS_ID', 'UP_AREA', 'geometry']]
# Add an additional column that converts gauge ID to HYBAS_ID in the streamflow df
streamflow_df['HYBAS_ID'] = streamflow_df['gauge_id'].str.replace('hybas_', '', regex=False).astype(int)
merge_df = pd.merge(hydrobasins_subset, streamflow_df, on='HYBAS_ID', how='inner')
merge_df.head()

,HYBAS_ID,UP_AREA,geometry,gauge_id,time,streamflow,estimated_rp
0,1121143370,143.0,"POLYGON ((40.97917 0.18333, 40.99076 0.18271, ...",hybas_1121143370,2020-05-01,0.648337,3.207878
1,1122081850,185.5,"POLYGON ((40.96667 0.21667, 40.95927 0.21728, ...",hybas_1122081850,2020-05-01,0.710088,3.019565
2,1121143780,279.3,"POLYGON ((40.98333 0.02083, 40.98333 0.025, 40...",hybas_1121143780,2020-05-01,1.452502,3.508826
3,1121143630,258.2,"POLYGON ((40.98333 0.04583, 40.98002 0.04668, ...",hybas_1121143630,2020-05-01,1.176055,3.596373
4,1122083890,561.1,"POLYGON ((41.02917 -0.16667, 41.02855 -0.16451...",hybas_1122083890,2020-05-01,2.576414,3.148140


#### Step 7: Plot the Maps

##### First just plotting streamflow

In [13]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
merge_df.plot(column='streamflow', cmap='Blues', legend=True, ax=ax)
ax.set_title(f"Streamflow on {selected_date}")
ax.axis('off')
plt.show()

##### Now plot by Return Period

In [15]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# Preprocess plotting RPs so values less than 2 become 0
merge_df['plot_rp'] = merge_df['estimated_rp'].apply(lambda x: x if x >= 2 else 0)

# Define RP boundaries
bounds = [0, 2, 5, 10, 50, 100, 200]

# Creatin a custom colormap (white for 0, and a continuous cmap for higher values)
base_cmap = plt.get_cmap('jet')
new_colors = ['#ffffff'] + [base_cmap(i) for i in range(base_cmap.N)]
cmap = mcolors.LinearSegmentedColormap.from_list("custom_jet", new_colors, N=256)

# Normalization
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N, extend='max')

merge_df.plot(
    column='plot_rp', cmap=cmap, norm=norm, linewidth=0.05, edgecolor='grey',
    legend=True, ax=ax,
    legend_kwds={
        'label': "Return Period (years)",
        'orientation': 'vertical',
        'ticks': bounds
    }
)
ax.set_title(f"Return Period on {selected_date}")
ax.axis('off')
plt.show()